In [ ]:
import importlib
import os
from pathlib import Path
import json

import numpy as np

import torch
import torch.nn as nn
import torch.utils.data

from scipy.spatial import distance
import matplotlib.pyplot as plt
from tqdm import tqdm

import pandas as pd
import random
from multiprocessing import Pool

In [9]:
detection_size = 25

In [10]:
torch.cuda.is_available()

True

In [42]:
image_folder = '/home/oqbrady/data/data/'
label_file_root = '/home/oqbrady/data/data/'
chips_path = '/home/oqbrady/data/data/chips/'
train_data_root = Path(image_folder) / 'train'
train_label_file = Path(label_file_root) / 'train.csv'
train_chips_path = Path(chips_path) / 'train'
val_data_root = Path(image_folder) / 'validation'
val_label_file = Path(label_file_root) / 'validation.csv'
val_chips_path = Path(chips_path) / 'validation'
train_chips = '/home/oqbrady/data/data/chips/train'
val_chips = '/home/oqbrady/data/data/chips/validation'

In [43]:
# val_label = pd.read_csv(val_label_file)

In [41]:
val_label['scene_id'].unique()

array(['264ed833a13b7f2av', '13dd786ee6c95e06v', '590dd08f71056cacv',
       '0d8ed29b0760dc59v', '2e80028071b89173v', '9a5aa7310c195f14v',
       'e40c9251a4fce150v', '3ceef682fbe4930av', '39268d50efe8dae9v',
       'a6073537a4ea5fb9v', 'cd4f6ff86f9f5991v', '4a97701b4bd81bf7v',
       'f298dbd78ef977d5v', 'cd36e75010a021f7v', '5e9a2c1bcf179e9bv',
       'fc2f57371370c521v', '128443d1e98e2839v', '94dca5e4de2edcf8v',
       '335f9a411884e9cbv', 'b1844cde847a3942v', '377410f6ab9824dfv',
       '36076e547370064ev', '9b89b9dcce7dc85ev', '4da9db72dea50504v',
       'acf76647ba3f524bv', 'c8bbfb37522617e9v', '9ea5029406691ce4v',
       '5c3d986db930f848v', 'f9eb760aaf6e798dv', '8204efcfe9f09f94v',
       'a1a21a222e244555v', '758991708403f218v', 'fe6a8d80fb5ebb8ev',
       '3fe00bf7beab8812v', '487b4884f467c94av', 'fc483682c621b54bv',
       '6a2b6ddecd398c6fv', '0157baf3866b2cf9v', '75c03770095c6d9ev',
       '0e309957cef7bdc8v', '7b7e837a7ac5a880v', '65d6f6baee882077v',
       '844545c00577

In [44]:
train_raw = pd.read_csv('/home/oqbrady/data/data/chips/train/train_chip_annotations.csv')
train_open = train_raw[~pd.isna(train_raw['is_vessel'])]
val_raw = pd.read_csv('/home/oqbrady/data/data/chips/validation/val_chip_annotations.csv')
val_open = val_raw[~pd.isna(val_raw['is_vessel'])]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,8,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
# for positives
train_group = train_open[train_open['source'] != 'background'].groupby(by=['chip_index', "scene_id"])
val_group = val_open[val_open['source'] != 'background'].groupby(by=['chip_index', "scene_id"])

In [46]:
val_raw['scene_id'].unique()

array(['7b7e837a7ac5a880v'], dtype=object)

In [19]:
save_path = '/home/oqbrady/data/data/sub_boxes/positives_v3/'

In [20]:
# os.mkdir('/home/oqbrady/data/data/sub_boxes/positives_v3/')

In [21]:
# fls = os.listdir('/home/oqbrady/data/data/sub_boxes/positives_v2')

In [22]:
# done = {}
# for i in fls:
#     s, c = i.split('_')[:2]
#     if s not in done:
#         done[s] = set()
#     done[s].add(c)

In [48]:
# labels = [class, length] # class is 0, nothing, 1 M.O., 2 vessel, 3 fishing
# def process_scene(train_img, labels, lengths, jitters, k):
k = 0
labels = []
lengths = []
jitters_x = []
jitters_y = []

scenes_tot = []
chips = []
nums = []

for train_img in tqdm(os.listdir(train_chips_path)):
    if "." in train_img:
        print(train_img)
        continue
    scences = os.listdir(chips_path + 'train/' + train_img + '/vh')
    scene_nums = [x.split("_")[0] for x in scences]
    for num in scene_nums:
        key = (int(num), train_img)
        if key not in train_group.groups:
#             print(key)
            continue
        detections = train_group.get_group(key)
        vh = np.load(chips_path + 'train/' + train_img + '/vh/' + str(num) + '_vh.npy')
        vv = np.load(chips_path + 'train/' + train_img + '/vv/' + str(num) + '_vv.npy')
        bay = np.load(chips_path + 'train/' + train_img + '/bathymetry/' + str(num) + '_bathymetry.npy')
        for i in range(detections.shape[0]):
            row = detections.iloc[i, :]['rows']
            col = detections.iloc[i, :]['columns']
            jitter_x = random.randint(-25,25)
            jitter_y = random.randint(-25,25)
            splice_1 = vh[row - detection_size + jitter_x: row + detection_size + jitter_x,
                          col - detection_size + jitter_y: col + detection_size + jitter_y]
            splice_2 = vv[row - detection_size + jitter_x: row + detection_size + jitter_x,
                          col - detection_size + jitter_y: col + detection_size + jitter_y]
            splice_3 = bay[row - detection_size + jitter_x: row + detection_size + jitter_x,
                           col - detection_size + jitter_y: col + detection_size + jitter_y]
            if splice_1.shape != (50, 50):
                continue
            jitters_x.append(jitter_x)
            jitters_y.append(jitter_y)
            labels.append(detections.iloc[i, :]['vessel_class'])
            lengths.append(detections.iloc[i, :]['vessel_length_m'])
            foldername = train_img + '_' + num + '_' + str(k)
            scenes_tot.append(train_img)
            chips.append(num)
            nums.append(k)
            try:
                os.mkdir(save_path + foldername)
                np.save(save_path + foldername + '/' + str(k) + '_vh.npy', splice_1)
                np.save(save_path + foldername + '/' + str(k) + '_vv.npy', splice_2)
                np.save(save_path + foldername + '/' + str(k) + '_bay.npy', splice_3)
                k += 1
            except:
                continue

 36%|██████████████████████████████▊                                                       | 199/555 [42:54<1:19:59, 13.48s/it]

train_chip_annotations.csv


100%|██████████████████████████████████████████████████████████████████████████████████████| 555/555 [2:03:34<00:00, 13.36s/it]


In [50]:
print(len(labels), len(lengths), len(jitters_x), len(jitters_y), len(nums), len(scenes_tot))

46620 46620 46620 46620 46620 46620


In [54]:
print("(hey)")

(hey)


In [52]:
df = pd.DataFrame({"labels" : labels, "lengths" : lengths, "jitter_x" : jitters_x, 'jitter_y' : jitters_y,
                   'scene' : scenes_tot, 'chip' : chips, 'num' : nums})

In [53]:
df.to_csv("labels_v4_train.csv")

In [55]:
df.shape

(46620, 7)

In [56]:
plt.imshow(viz[60], cmap='bone')

NameError: name 'viz' is not defined

In [80]:
images = []
# labels = [class, length] # class is 0, nothing, 1 M.O., 2 vessel, 3 fishing
labels = []
jitters = []
scenes = []
chips = [
]
for train_img in tqdm(os.listdir(train_data_root)[:25]):
    scences = os.listdir(chips_path + 'train/' + train_img + '/vh')
    scene_nums = [x.split("_")[0] for x in scences]
    for num in scene_nums:
        vh = np.load(chips_path + 'train/' + train_img + '/vh/' + str(num) + '_vh.npy')
        vv = np.load(chips_path + 'train/' + train_img + '/vv/' + str(num) + '_vv.npy')
        bay = np.load(chips_path + 'train/' + train_img + '/bathymetry/' + str(num) + '_bathymetry.npy')
        row = random.randint(25,75)
        col = random.randint(25,75)
        splice_1 = vh[row - detection_size: row + detection_size, col - detection_size: col + detection_size]
        splice_2 = vv[row - detection_size: row + detection_size, col - detection_size: col + detection_size]
        splice_3 = bay[row - detection_size: row + detection_size, col - detection_size: col + detection_size]
        if img.shape != ():
            continue
            print(img.shape)
            print(row - detection_size, row + detection_size, col - detection_size, col + detection_size)
        images.append(img)
        labels.append(0)

  0%|                                                                                                           | 0/25 [00:00<?, ?it/s]


NameError: name 'img' is not defined

In [ ]:
k = 0
labels = []
lengths = []
jitters = []
for train_img in tqdm(os.listdir(train_chips_path)):
    if "." in train_img:
        print(train_img)
        continue
    scences = os.listdir(chips_path + 'train/' + train_img + '/vh')
    scene_nums = [x.split("_")[0] for x in scences]
    for num in scene_nums:
        key = (int(num), train_img)
        if key not in train_group.groups:
#             print(key)
            continue
        detections = train_group.get_group(key)
#         vh = np.load(chips_path + 'train/' + train_img + '/vh/' + str(num) + '_vh.npy')
#         vv = np.load(chips_path + 'train/' + train_img + '/vv/' + str(num) + '_vv.npy')
#         bay = np.load(chips_path + 'train/' + train_img + '/bathymetry/' + str(num) + '_bathymetry.npy')
        for i in range(detections.shape[0]):
            row = detections.iloc[i, :]['rows']
            col = detections.iloc[i, :]['columns']
            jitter_x = random.randint(-25,25)
            jitter_y = random.randint(-25,25)
#             splice_1 = vh[row - detection_size + jitter_x: row + detection_size + jitter_x,
#                           col - detection_size + jitter_y: col + detection_size + jitter_y]
#             splice_2 = vv[row - detection_size + jitter_x: row + detection_size + jitter_x,
#                           col - detection_size + jitter_y: col + detection_size + jitter_y]
#             splice_3 = bay[row - detection_size + jitter_x: row + detection_size + jitter_x,
#                            col - detection_size + jitter_y: col + detection_size + jitter_y]
            if splice_1.shape != (50, 50):
                continue
            jitters.append([jitter_x, jitter_y])
            labels.append(detections.iloc[i, :]['vessel_class'])
            lengths.append(detections.iloc[i, :]['vessel_length_m'])
            foldername = train_img + '_' + num + '_' + str(k)
            try:
                os.mkdir(save_path + foldername)
                np.save(save_path + foldername + '/' + str(k) + '_vh.npy', splice_1)
                np.save(save_path + foldername + '/' + str(k) + '_vv.npy', splice_2)
                np.save(save_path + foldername + '/' + str(k) + '_bay.npy', splice_3)
                k += 1
            except:
                continue
df = pd.DataFrame({"labels" : labels, "lengths" : lengths, "jitters" : jitters})
df.to_csv("labels_v2_train.csv")